<a href="https://colab.research.google.com/github/gothammered/2022-2-Python/blob/main/HW1_SubwayNavigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hello world

In [1]:
"""
Using xlrd is a good way to read excel file.
However, because not only the package is no longer maintained but also can only read xls file, which is also deprecated, this code used pandas instead.
(Please refer to https://github.com/python-excel/xlrd for further information regarding xlrd package)
"""

# Instead of
# import xlrd
# this code will
import pandas as pd
# because of the reasons mentioned above.



"""
Still,

# Read data file
data = xlrd.open_workbook("simplified_subway_info_english.xlsx")
data = data.sheet_by_name('Sheet1')

# Store the loaded book object as a string list in subwayStation variable
subwayStation = []
for line in range(4) :
    cur = [x for x in data.col_values(line) if x]    
    subwayStation.append(cur[1:])

"""

[['Soyosan', 'Dongducheon', 'Bosan', 'Dongducheon jungang', 'Jihaeng', 'Deokjeong', 'Deokgye', 'Yangju', 'Nogyang', 'Ganeung', 'Uijeongbu', 'Hoeryong', 'Mangwolsa', 'Dobongsan', 'Dobong', 'Banghak', 'Chang-dong', 'Nokcheon', 'Wolgye', 'Kwangwoon Univ.', 'Seokgye', 'Sinimun', 'Hankuk Univ. of Foreign Studies', 'Hoegi', 'Cheongnyangni', 'Jegi-dong', 'Sinseol-dong', 'Dongmyo', 'Dongdaemun', 'Jongno 5(o)-ga', 'Jongno 3(sam)-ga', 'Jonggak', 'City Hall', 'Seoul Station', 'Namyeong', 'Yongsan', 'Noryangjin', 'Daebang', 'Singil', 'Yeongdeungpo', 'Sindorim', 'Guro', 'Guil', 'Gaebong', 'Oryu-dong', 'Onsu', 'Yeokgok', 'Sosa', 'Bucheon', 'Jung-dong', 'Songnae', 'Bugae', 'Bupyeong', 'Baegun', 'Dongam', 'Ganseok', 'Juan', 'Dohwa', 'Jemulpo', 'Dowon', 'Dongincheon', 'Incheon'], ['Sindorim', 'Daerim', 'Guro Digital Complex', 'Sindaebang', 'Sillim', 'Bongcheon', 'Seoul Nat`l Univ.', 'Nakseongdae', 'Sadang', 'Bangbae', 'Seocho', 'Seoul Nat`l Univ. of Education', 'Gangnam', 'Yeoksam', 'Seolleung', 'Samse